# Строим точечный график с гиперссылками на сайты

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install bokeh
import bokeh.models as bm, bokeh.plotting as pl, bokeh.palettes as palettes
from bokeh.io import output_notebook
output_notebook()

    100% |################################| 16.2MB 120kB/s eta 0:00:01   21% |######                          | 3.5MB 40.4MB/s eta 0:00:01
    100% |################################| 276kB 3.5MB/s eta 0:00:01
  Running setup.py bdist_wheel for bokeh ... done
  Stored in directory: /root/.cache/pip/wheels/1a/a0/ec/d46994ac427b4879969dd780cf422bd3a0886fb85f481dd064
  Running setup.py bdist_wheel for PyYAML ... done
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
Successfully built bokeh PyYAML
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Loading BokehJS ...

In [3]:
def draw_vectors(x, y, radius=10, alpha=0.25, classes=None,
                 width=600, height=400, show=True, tokens=None):
    
    if classes is None:
        color = ["blue"] * len(x)
    else:
        n = np.unique(classes)
        palette = palettes.viridis(len(n))
        indx = {n[i]:i for i in range(len(n))}
        color = [palette[indx[i]] for i in classes]
    
    if isinstance(color, str): color = [color] * len(x)
    if classes is None:
        data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, "tokens": tokens })
    else:
        data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, 'classes':classes, "tokens": tokens })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    if classes is None:
        fig.circle('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)
    else:
        fig.circle('x', 'y', size=radius, color='color', alpha=alpha, source=data_source, legend='classes')

    fig.add_tools(bm.HoverTool(tooltips=[("tokens", "@" + "tokens")]))
    
    url = "https://@tokens"
    fig.add_tools(bm.TapTool(callback=bm.OpenURL(url=url)))
    if not classes is None:
        fig.legend.location = "top_left"
    if show: pl.show(fig)
    return fig

In [4]:
df = pd.read_csv('vectors_43.csv', sep='\t')

Считываем файл. Сортируем по классам, записываем вектора. Eval превращает строку в лист с вектором.

In [5]:
df = df.sort_values('lab')
lda_vec_, lsi_vec_, doc2vec_, y_ = df['lda_vector'].apply(lambda x: eval(x)),\
    df['lsi_v2_vector'].apply(lambda x: eval(x)), df['doc2vec_v2_vector'].apply(lambda x: eval(x)),\
    df['lab'].apply(lambda r:int(r)).tolist()
print(len(lda_vec_), len(lsi_vec_), len(doc2vec_))

2740 2740 2740


In [6]:
a = np.array(list(lda_vec_ + lsi_vec_ + doc2vec_))
lda_vec = np.array(list(lda_vec_))
lsi_vec = np.array(list(lsi_vec_))
doc2vec = np.array(list(doc2vec_))
lab = y_#np.array(list(y_))

Понижаем размерность.

In [7]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
g1 = pca.fit_transform(lda_vec)
g2 = pca.fit_transform(lsi_vec)
g3 = pca.fit_transform(doc2vec)
g = pca.fit_transform(a)

Строим график и передаем точкам домены

In [8]:
# %matplotlib inline
data = df['domain']
x, y = g1[:,0], g1[:,1]
color=df['lab']
draw_vectors(x,y, classes=color,alpha=0.7, tokens=data)

Figure(id='1003', ...)